In [1]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
import pandas as pd
import os
#custom
from cnn_utils import *

#tensorflow
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow_datasets as tfds
import tensorflow_text as text
    
from dotenv import load_dotenv 

### Introduction

Game state

1. The game starts on an empty grid
2. Every agent adds one X (1) or O (-1) when its their turn
3. The game ends when one player has three games in a row, column, or diagonally, or
4. there are no more open fields

Rewards

1. if one player wins, this player gets the reward
2. if it is a draw, both players get no points.
   I.e., a loss = a draw

Goal
Win the game. Not more sophisticated strategies

Policy
Here a policy is a rule that tells the player what move to make for every state of the game. 
For each considered policy an estimate of its winning probability would be obtained by playing 
some number of games against the opponent    

In [2]:
from TicTacToe import Player as p
from TicTacToe import HumanPlayer as hp
from TicTacToe import State as s


In [6]:
# training
p1 = p.Player("p1")
p2 = p.Player("p2")

st = s.State(p1, p2)
print("Playing a few training rounds...")
st.play(5000)
#store the success policies of each agent
p1.savePolicy()
p2.savePolicy()

Playing a few training rounds...
Rounds 0
P2 wins
-------------
| x |   | o | 
-------------
|   |   | o | 
-------------
| x | x | o | 
-------------
P2 wins
-------------
|   | o | x | 
-------------
| x | o | o | 
-------------
| x | o | x | 
-------------
P1 wins
-------------
|   |   | x | 
-------------
| o | x | o | 
-------------
| x | o | x | 
-------------
P1 wins
-------------
| x | x |   | 
-------------
|   | x | o | 
-------------
| o | o | x | 
-------------
P1 wins
-------------
| x | x |   | 
-------------
|   | x | o | 
-------------
| o | o | x | 
-------------
P1 wins
-------------
| x | x | x | 
-------------
| o | o | x | 
-------------
| o | x | o | 
-------------
P1 wins
-------------
| x |   |   | 
-------------
| o | x | o | 
-------------
| x | o | x | 
-------------
P2 wins
-------------
|   | o | x | 
-------------
| x | o | o | 
-------------
| x | o | x | 
-------------
P1 wins
-------------
|   |   | x | 
-------------
| o | x | o | 
-------------
| x | 

P1 wins
-------------
|   |   | x | 
-------------
|   |   | x | 
-------------
| o | o | x | 
-------------
P1 wins
-------------
|   | x | x | 
-------------
|   | o | x | 
-------------
| o | o | x | 
-------------
P1 wins
-------------
| o | x | x | 
-------------
| o | x | o | 
-------------
| x | o | x | 
-------------
P2 wins
-------------
| x | x |   | 
-------------
| o | o | o | 
-------------
| x | o | x | 
-------------
Draw
-------------
| x | o | x | 
-------------
| x | o | o | 
-------------
| o | x | x | 
-------------
P2 wins
-------------
|   | o | x | 
-------------
| x | o | o | 
-------------
| x | o | x | 
-------------
P2 wins
-------------
| x |   |   | 
-------------
| x | x |   | 
-------------
| o | o | o | 
-------------
Draw
-------------
| x | o | x | 
-------------
| x | o | o | 
-------------
| o | x | x | 
-------------
P1 wins
-------------
|   |   |   | 
-------------
|   | o | o | 
-------------
| x | x | x | 
-------------
Draw
-------------
| x | 

P2 wins
-------------
| x | o |   | 
-------------
|   | o |   | 
-------------
| x | o | x | 
-------------
P1 wins
-------------
|   |   |   | 
-------------
| o | o |   | 
-------------
| x | x | x | 
-------------
P1 wins
-------------
|   |   | x | 
-------------
| o | o | x | 
-------------
| o | x | x | 
-------------
P2 wins
-------------
|   |   | o | 
-------------
| x | o |   | 
-------------
| o | x | x | 
-------------
P2 wins
-------------
| x | o |   | 
-------------
|   | o |   | 
-------------
| x | o | x | 
-------------
P2 wins
-------------
| x |   | o | 
-------------
| x | o | x | 
-------------
| o | x | o | 
-------------
P1 wins
-------------
| o |   |   | 
-------------
|   | o |   | 
-------------
| x | x | x | 
-------------
P1 wins
-------------
|   |   | o | 
-------------
|   | o |   | 
-------------
| x | x | x | 
-------------
P2 wins
-------------
|   | x | x | 
-------------
| o | o | o | 
-------------
| o | x | x | 
-------------
P2 wins
-----------

P1 wins
-------------
| o |   | x | 
-------------
|   | o | x | 
-------------
| o | x | x | 
-------------
P1 wins
-------------
|   |   | x | 
-------------
| o | x | o | 
-------------
| x | o | x | 
-------------
P1 wins
-------------
| x | x | x | 
-------------
|   | o | o | 
-------------
|   | o | x | 
-------------
P2 wins
-------------
|   | x | x | 
-------------
| o | o | o | 
-------------
| o | x | x | 
-------------
P2 wins
-------------
| x |   | o | 
-------------
|   | o |   | 
-------------
| o | x | x | 
-------------
P1 wins
-------------
| x | o | x | 
-------------
|   |   | x | 
-------------
| o | o | x | 
-------------
Draw
-------------
| o | x | x | 
-------------
| x | o | o | 
-------------
| o | x | x | 
-------------
Draw
-------------
| o | x | x | 
-------------
| x | o | o | 
-------------
| o | x | x | 
-------------
P2 wins
-------------
| o |   | x | 
-------------
| x | o | o | 
-------------
| x | x | o | 
-------------
P2 wins
-------------
| x

P1 wins
-------------
| o |   | x | 
-------------
|   | o | x | 
-------------
| o | x | x | 
-------------
Draw
-------------
| o | x | x | 
-------------
| x | o | o | 
-------------
| o | x | x | 
-------------
Draw
-------------
| o | x | x | 
-------------
| x | o | o | 
-------------
| o | x | x | 
-------------
P1 wins
-------------
| x | o |   | 
-------------
| x |   | o | 
-------------
| x | x | o | 
-------------
P2 wins
-------------
|   | o | o | 
-------------
| x | o | x | 
-------------
| x | o | x | 
-------------
P2 wins
-------------
|   | o | o | 
-------------
| x | o | x | 
-------------
| o | x | x | 
-------------
P1 wins
-------------
| o |   |   | 
-------------
|   | o |   | 
-------------
| x | x | x | 
-------------
P1 wins
-------------
| o |   | x | 
-------------
|   | x | o | 
-------------
| x | x | o | 
-------------
Draw
-------------
| o | o | x | 
-------------
| x | x | o | 
-------------
| o | x | x | 
-------------
P2 wins
-------------
| o | 

...

-------------
|   | x | o | 
-------------
| x | o | o | 
-------------
| x | x | o | 
-------------
P2 wins
-------------
| o | o | x | 
-------------
| x | o |   | 
-------------
| x | x | o | 
-------------
P2 wins
-------------
| o |   |   | 
-------------
| x | o |   | 
-------------
| x | x | o | 
-------------
P2 wins
-------------
| o | x |   | 
-------------
|   | o |   | 
-------------
| x | x | o | 
-------------
P2 wins
-------------
| x | o | x | 
-------------
| x | x |   | 
-------------
| o | o | o | 
-------------
P1 wins
-------------
| x | x | x | 
-------------
| o | o |   | 
-------------
| o | x |   | 
-------------
P1 wins
-------------
| x | x | x | 
-------------
| o | o |   | 
-------------
| o | x |   | 
-------------
Draw
-------------
| x | x | o | 
-------------
| o | o | x | 
-------------
| x | o | x | 
-------------
P1 wins
-------------
| o |   |   | 
-------------
|   | o |   | 
-------------
| x | x | x | 
-------------
P1 wins
-------------
| x |   

P1 wins
-------------
|   |   | x | 
-------------
|   | o | x | 
-------------
|   | o | x | 
-------------
P1 wins
-------------
| x |   |   | 
-------------
| o | x |   | 
-------------
| o |   | x | 
-------------
P1 wins
-------------
| x | x | o | 
-------------
| x | o | x | 
-------------
| x | o | o | 
-------------
Draw
-------------
| o | x | o | 
-------------
| x | o | x | 
-------------
| x | o | x | 
-------------
P1 wins
-------------
|   |   | o | 
-------------
|   | o |   | 
-------------
| x | x | x | 
-------------
P2 wins
-------------
| x | x |   | 
-------------
| o | o | o | 
-------------
|   |   | x | 
-------------
P1 wins
-------------
|   |   | x | 
-------------
|   | o | x | 
-------------
| o |   | x | 
-------------
P1 wins
-------------
|   |   |   | 
-------------
|   | o | o | 
-------------
| x | x | x | 
-------------
P1 wins
-------------
| x | x | x | 
-------------
|   | o | o | 
-------------
|   | o | x | 
-------------
P1 wins
-------------


Draw
-------------
| x | x | o | 
-------------
| o | o | x | 
-------------
| x | o | x | 
-------------
P1 wins
-------------
| x |   |   | 
-------------
| x | o | o | 
-------------
| x | o | x | 
-------------
P1 wins
-------------
|   |   |   | 
-------------
| o | o |   | 
-------------
| x | x | x | 
-------------
P2 wins
-------------
| o | o | o | 
-------------
|   | x | x | 
-------------
|   |   | x | 
-------------
P1 wins
-------------
| o | o | x | 
-------------
| x | x | x | 
-------------
| o | o | x | 
-------------
P1 wins
-------------
|   |   | x | 
-------------
|   | x |   | 
-------------
| x | o | o | 
-------------
P2 wins
-------------
|   | o | o | 
-------------
| x | o | x | 
-------------
| x | o | x | 
-------------
P2 wins
-------------
|   | x | o | 
-------------
| x | o | x | 
-------------
| o | o | x | 
-------------
P1 wins
-------------
| x |   |   | 
-------------
| o | x |   | 
-------------
| o |   | x | 
-------------
P1 wins
-------------


P2 wins
-------------
| o | o | x | 
-------------
| x | o |   | 
-------------
| x | x | o | 
-------------
Draw
-------------
| x | o | x | 
-------------
| o | x | x | 
-------------
| o | x | o | 
-------------
P1 wins
-------------
|   | o | x | 
-------------
| o |   | x | 
-------------
| o | x | x | 
-------------
P1 wins
-------------
| x |   |   | 
-------------
| x | o | o | 
-------------
| x | x | o | 
-------------
P2 wins
-------------
| o | o | o | 
-------------
|   | x | x | 
-------------
| o | x | x | 
-------------
P1 wins
-------------
|   | x | o | 
-------------
| o | o |   | 
-------------
| x | x | x | 
-------------
P1 wins
-------------
| x | x | x | 
-------------
|   | o | o | 
-------------
| o |   | x | 
-------------
P2 wins
-------------
| o | x | o | 
-------------
| x | o |   | 
-------------
| o | x | x | 
-------------
P1 wins
-------------
| o | o | x | 
-------------
|   |   | x | 
-------------
| o | x | x | 
-------------
P1 wins
-------------


P2 wins
-------------
| x |   | o | 
-------------
|   |   | o | 
-------------
| x | x | o | 
-------------
P2 wins
-------------
|   |   | x | 
-------------
| o | o | o | 
-------------
| x |   | x | 
-------------
P1 wins
-------------
| x |   | x | 
-------------
| o | x |   | 
-------------
| x | o | o | 
-------------
Draw
-------------
| o | x | o | 
-------------
| x | x | o | 
-------------
| x | o | x | 
-------------
Draw
-------------
| x | x | o | 
-------------
| o | o | x | 
-------------
| x | o | x | 
-------------
Draw
-------------
| x | x | o | 
-------------
| o | x | x | 
-------------
| x | o | o | 
-------------
P2 wins
-------------
| o | x | x | 
-------------
|   | o | o | 
-------------
| x | x | o | 
-------------
Draw
-------------
| x | o | x | 
-------------
| o | o | x | 
-------------
| x | x | o | 
-------------
P1 wins
-------------
| x | x | o | 
-------------
| o | x | o | 
-------------
| x | o | x | 
-------------
P2 wins
-------------
| x |   |

In [4]:
# play with human
#p1 = p.Player("computer", exp_rate=0)
#p1.loadPolicy("policy_p1")

#p2 = hp.HumanPlayer("human")

#st = s.State(p1, p2)
#st.play2()